In [2]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.tsa.statespace.sarimax as sarima
import statsmodels.tools.eval_measures as fa
import arch.univariate as arch

In [4]:
# 2. Data Downloading or Reading

# Yahoo Finance
spx = web.DataReader('^GSPC', data_source='yahoo', start='2007-01-01', end='2018-06-30')
vix = web.DataReader('^VIX', data_source='yahoo', start='2007-01-01', end='2018-06-30')
spxohlc = spx[['Open', 'High', 'Low', 'Close']]
spxohlc.columns = ['SPX.Open', 'SPX.High', 'SPX.Low', 'SPX.Close']
vixc = vix['Close']
hdata = spxohlc.join(vixc)
hdata = hdata.dropna()
hdata.columns = ['SPX.Open', 'SPX.High', 'SPX.Low', 'SPX.Close', 'VIX.Close']

# Data Reading
#hdata = pd.read_csv("Data//Volatility-Trading-Analysis-Data.txt", index_col='Date', parse_dates=True)

In [6]:
hdata.head()

,SPX.Open,SPX.High,SPX.Low,SPX.Close,VIX.Close
Date,,,,,
2007-01-03,1418.030029,1429.420044,1407.859985,1416.599976,12.04
2007-01-04,1416.599976,1421.839966,1408.430054,1418.339966,11.51
2007-01-05,1418.339966,1418.339966,1405.750000,1409.709961,12.14
2007-01-08,1409.260010,1414.979980,1403.969971,1412.839966,12.00
2007-01-09,1412.839966,1415.609985,1405.420044,1412.109985,11.91


In [8]:
rspx = (np.log(hdata.loc[:, 'SPX.Close'] / hdata.loc[:, 'SPX.Close'].shift(1))) * 100
hgarchm = arch.ZeroMean(y=rspx.shift(21)['2007-01-03':'2016-12-30'].dropna(),
                        volatility=arch.GARCH(p=1, o=0, q=1, power=2.0),
                        distribution=arch.Normal()).fit()
hgarchf = arch.ZeroMean(y=rspx.shift(21)['2017-01-03':'2018-06-29'].dropna(),
                        volatility=arch.GARCH(p=1, o=0, q=1, power=2.0),
                        distribution=arch.Normal()).fix(params=hgarchm.params)
hgarchf = hgarchf.conditional_volatility*np.sqrt(252)

Iteration:      1,   Func. Count:      5,   Neg. LLF: 3584.67050567345
Iteration:      2,   Func. Count:     12,   Neg. LLF: 3579.199655098204
Iteration:      3,   Func. Count:     19,   Neg. LLF: 3578.134829217487
Iteration:      4,   Func. Count:     25,   Neg. LLF: 3577.441231691666
Iteration:      5,   Func. Count:     32,   Neg. LLF: 3577.3960714923924
Iteration:      6,   Func. Count:     38,   Neg. LLF: 3577.0091128354957
Iteration:      7,   Func. Count:     43,   Neg. LLF: 3576.9822088989295
Iteration:      8,   Func. Count:     48,   Neg. LLF: 3576.98194930397
Iteration:      9,   Func. Count:     53,   Neg. LLF: 3576.9819482656058
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 3576.981948266075
            Iterations: 9
            Function evaluations: 53
            Gradient evaluations: 9


In [9]:
hgarchm

                       Zero Mean - GARCH Model Results                        
Dep. Variable:              SPX.Close   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:               -3576.98
Distribution:                  Normal   AIC:                           7159.96
Method:            Maximum Likelihood   BIC:                           7177.43
                                        No. Observations:                 2496
Date:                Wed, Apr 15 2020   Df Residuals:                     2493
Time:                        15:29:42   Df Model:                            3
                              Volatility Model                              
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
omega          0.0317  7.744e-03      4.094  4.231e-05 [1.